In [ ]:
# Install required libraries
!pip install -q transformers accelerate peft datasets trl bitsandbytes
!pip install -q git+https://github.com/huggingface/peft.git

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 348.0/348.0 kB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.5/491.5 kB 23.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.1/76.1 MB 14.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 17.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 110.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 88.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 55.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
from huggingface_hub import notebook_login
notebook_login()

In [ ]:
from datasets import load_dataset, Dataset
import pandas as pd
from sklearn.model_selection import train_test_split

# Load dataset
ds = load_dataset("LawInformedAI/claudette_tos")
df = pd.DataFrame(ds["train"])

# Sample examples per class
num_samples = 1000
anomalous = df[df['label'] == 1].sample(n=num_samples, random_state=42)
normal = df[df['label'] == 0].sample(n=num_samples, random_state=42)

# Combine and shuffle
balanced_df = pd.concat([anomalous, normal]).sample(frac=1, random_state=42).reset_index(drop=True)

# First split: train (80%) and temp (20%)
train_df, temp_df = train_test_split(
    balanced_df,
    test_size=0.2,
    stratify=balanced_df['label'],
    random_state=42
)

# Second split: validation (10%) and test (10%) from temp
val_df, test_df = train_test_split(
    temp_df,
    test_size=0.5,
    stratify=temp_df['label'],
    random_state=42
)

# Convert to Hugging Face Datasets
train_dataset = Dataset.from_pandas(train_df)
val_dataset = Dataset.from_pandas(val_df)
test_dataset = Dataset.from_pandas(test_df)

# Check distributions
print("Train distribution:\n", train_df['label'].value_counts())
print("Validation distribution:\n", val_df['label'].value_counts())
print("Test distribution:\n", test_df['label'].value_counts())


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/382 [00:00<?, ?B/s]

(…)-00000-of-00001-a8de7efe0da36666.parquet:   0%|          | 0.00/856k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/9414 [00:00<?, ? examples/s]

Train distribution:
 label
1    800
0    800
Name: count, dtype: int64
Validation distribution:
 label
1    100
0    100
Name: count, dtype: int64
Test distribution:
 label
1    100
0    100
Name: count, dtype: int64


In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from peft import get_peft_model, LoraConfig, TaskType
import torch

model_name = "meta-llama/Llama-2-7b-hf"

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16,
    llm_int8_enable_fp32_cpu_offload=True
)

tokenizer = AutoTokenizer.from_pretrained(model_name, token=True)
tokenizer.pad_token = tokenizer.eos_token  # causal LM fix

base_model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map="auto",
    quantization_config=bnb_config,
    torch_dtype=torch.float16,
    trust_remote_code=True,
    token=True
)

# LoRA config
lora_config = LoraConfig(
    r=8,
    lora_alpha=16,
    target_modules=["q_proj", "v_proj"],
    lora_dropout=0.05,
    bias="none",
    task_type=TaskType.CAUSAL_LM
)

# Apply LoRA to base model
model = get_peft_model(base_model, lora_config)
model.print_trainable_parameters()


tokenizer_config.json:   0%|          | 0.00/776 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/609 [00:00<?, ?B/s]

...

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/188 [00:00<?, ?B/s]

trainable params: 4,194,304 || all params: 6,742,609,920 || trainable%: 0.0622


In [ ]:
# Step 1: Format prompts and targets
def format(example):
    prompt = f"<s>[CLAUSE]: {example['text']} \n[Is this anomalous?]:"
    answer = " Yes" if example['label'] == 1 else " No"
    full_text = prompt + answer
    return {"text": full_text, "prompt": prompt}

train_dataset = train_dataset.map(format)
val_dataset = val_dataset.map(format)
test_dataset = test_dataset.map(format)

# Step 2: Tokenization + label masking for causal LM
def tokenize(example):
    # Tokenize full input
    full_input = tokenizer(example["text"], truncation=True, padding="max_length", max_length=256)

    # Compute where the prompt ends (tokens to mask with -100)
    prompt_len = len(tokenizer(example["prompt"], add_special_tokens=False)["input_ids"])

    # Copy input IDs and mask prompt part in labels
    labels = full_input["input_ids"][:]
    labels[:prompt_len] = [-100] * prompt_len
    full_input["labels"] = labels

    return full_input

# Apply tokenization to all datasets and remove extra columns
train_dataset = train_dataset.map(tokenize, remove_columns=train_dataset.column_names)
val_dataset = val_dataset.map(tokenize, remove_columns=val_dataset.column_names)
test_dataset = test_dataset.map(tokenize, remove_columns=test_dataset.column_names)


Map:   0%|          | 0/1600 [00:00<?, ? examples/s]

Map:   0%|          | 0/200 [00:00<?, ? examples/s]

Map:   0%|          | 0/200 [00:00<?, ? examples/s]

Map:   0%|          | 0/1600 [00:00<?, ? examples/s]

Map:   0%|          | 0/200 [00:00<?, ? examples/s]

Map:   0%|          | 0/200 [00:00<?, ? examples/s]

In [ ]:
from transformers import TrainingArguments
from trl import SFTTrainer

training_args = TrainingArguments(
    output_dir="./lora-llama-binary-classifier",
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    gradient_accumulation_steps=2,
    logging_dir="./logs",
    logging_steps=10,
    save_strategy="epoch",
    eval_strategy="epoch",
    num_train_epochs=1,
    learning_rate=2e-4,
    bf16=True,  # Use bf16 if your GPU supports it; otherwise use fp16=True
    load_best_model_at_end=True,
    save_total_limit=1,
    report_to="none"
)

trainer = SFTTrainer(
    model=model,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    args=training_args,
)
trainer.train()


Truncating train dataset:   0%|          | 0/1600 [00:00<?, ? examples/s]

Truncating eval dataset:   0%|          | 0/200 [00:00<?, ? examples/s]

No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


Epoch,Training Loss,Validation Loss


In [ ]:
import torch
import re
from tqdm import tqdm

device = "cuda" if torch.cuda.is_available() else "cpu"

model.eval()
predictions = []
true_labels = ["Yes" if label == 1 else "No" for label in test_df["label"].tolist()]

print("\nSample Predictions:\n")

for i, row in tqdm(test_df.iterrows(), total=len(test_df)):
    clause = row["text"]
    label = "Yes" if row["label"] == 1 else "No"

    prompt = f"<s>[CLAUSE]: {clause} \n[Is this anomalous?]:"
    inputs = tokenizer(prompt, return_tensors="pt").to(device)

    with torch.no_grad():
        outputs = model.generate(
            input_ids=inputs["input_ids"],
            attention_mask=inputs["attention_mask"],
            max_new_tokens=10,
            eos_token_id=tokenizer.eos_token_id,
            do_sample=True,
            temperature=0.7,
            top_p=0.9
        )

    output_text = tokenizer.decode(outputs[0], skip_special_tokens=True)

    # Extract answer
    match = re.search(r"\[Is this anomalous\?\]:(.*)", output_text)
    if match:
        answer = match.group(1).strip().lower()
        pred = "Yes" if answer.startswith("yes") else "No"
    else:
        pred = "Unknown"

    predictions.append(pred)

    # Print a few samples
    if i < 5:
        print(f"\nClause: {clause[:150]}{'...' if len(clause) > 150 else ''}")
        print(f"Model Output: {output_text}")
        print(f"Prediction: {pred} | Ground Truth: {label}")
        print("-" * 80)


In [ ]:
from sklearn.metrics import accuracy_score, classification_report

accuracy = accuracy_score(true_labels, predictions)
print(f"Test Accuracy: {accuracy:.4f}\n")

print("Classification Report:")
print(classification_report(true_labels, predictions))
